In [ ]:
# Processor requirements
# The first two cells must be code cells, inpath and outpath define the scan file location and output location
# inpath = "/dls/i16/data/2024/cm37262-15/1068022.nxs"  # Merlin
inpath = "/dls/i16/data/2024/cm37262-15/1068020.nxs"  # Pilatus

In [ ]:
outpath = ""

# I16 Detector Position Calibration

Calibrate the detector position using the direct beam and generate the geometry.xml file.

## Instructions

```
pos x1 0  # fast shutter closed
pos atten 255
pos delta 0
# Activate manual override of fast shutter in EPICS
pos pils 1  # check intensity level, adjust atten
pos x1 1
scan gam -0.8 1 0.2 delta -1 1.5 0.5 calibrate_detector pil3_100k 1
# or for QuadMerlin detector:
scan gam -0.15 0.15 0.05 delta -0.15 0.15 0.05 calibrate_detector merlin 1
```

Calibration files `script/pilatus_calibration/geometry_{detector}.xml` and `geometry_{detector}_{date}.xml` will be generated.

In [ ]:
# %matplotlib ipympl  # add to get interactive plots
# %config Completer.use_jedi = False  # add to make autocomplete work

import sys, os
import typing
import datetime
import xml.etree.ElementTree as ET

import ipywidgets as widgets
from IPython.display import display, Markdown, Latex

import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
import hdfmap

## Functions

In [ ]:
# Functions for processing data

def remove_hot_pixels(image_volume: np.ndarray) -> np.ndarray:
    """
    Find static pixels and set to 0
    image_volume must be [idx, i, j] where i,j are the image axes
    """
    live_pixels = np.sum(np.abs(np.diff(image_volume, axis=0)), axis=0)
    live_value = np.mean(live_pixels)
    for n in range(len(image_volume)):
        image_volume[n, live_pixels < live_value] = 0
    return image_volume


def rebin_image(image: np.ndarray, step: tuple[int, int] = (4, 4)) -> np.ndarray:
    """
    im = rebin_image(image, (di, dj))
    Rebins an image array, taking the average of adjacent pixels
    """
    
    i, j = image.shape
    di, dj = step
    
    # remove trailing elements
    image2 = image[:i - i % di, :j - j % dj]
    
    # sum pixels
    ct = 0.0
    imsum = np.zeros(list(np.array(image2.shape)//np.array(step)))
    for n in range(di):
        for m in range(dj):
            ct+=1.0
            imsum += image2[n::di, m::dj]
    # Take the average
    return imsum / ct


def peak_search(flat_data: np.ndarray) -> np.ndarray:
    """
    Finds brightest pixel in each image of an image stack

    :params flat_data: [nxixj] array where n in the scan direction, ixj is the image size
    :returns: [i, j] * n array of pixel positions
    """
    
    # Remove hot pixels
    flat_data = remove_hot_pixels(flat_data)

    AV = 4  # average pixels
    RG = 5  # half-ROI around max of average
            
    peaks = []
    for im in flat_data:
        im2 = rebin_image(im, (AV, AV))  # average adjacent pixels to reduce likelyhood of picking hot pixel
        i, j = np.unravel_index(im2.argmax(),im2.shape) # max pixel within the region
        i *= 4; j *= 4  # recover index of original array
        # peak region
        im3 = im[i-RG:i+RG, j-RG:j+RG]
        i2, j2 = np.unravel_index(im3.argmax(), im3.shape)
        peaks += [(i2 + i - 5, j2 + j - 5)]
    peaks = np.array(peaks)
    return peaks

In [ ]:
# Functions from dpcalib.py

def cs(degrees: float) -> tuple[float, float]:
    r = np.deg2rad(degrees)
    return np.cos(r), np.sin(r)


def rotate_x(degrees: float) -> np.ndarray:
    c, s = cs(degrees)
    return np.array([[1, 0, 0], [0, c, -s], [0, s, c]])


def rotate_y(degrees: float) -> np.ndarray:
    c, s = cs(degrees)
    return np.array([[c, 0, s], [0, 1, 0], [-s, 0, c]])


def rotate_z(degrees: float) -> np.ndarray:
    c, s = cs(degrees)
    return np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])


class Params(typing.NamedTuple):
    x_rot: float
    y_rot: float
    z_rot: float
    x_pos: float
    y_pos: float
    z_pos: float


def orient_detector(params: Params, delta: float = 0, gamma: float = 0) -> tuple[np.ndarray, np.ndarray]:
    '''Orient the detector with three Euler angles and two diffractometer circles
    Return transformation matrix from detector frame to laboratory frame and
    also diffractometer rotations
    '''
    a,b,c = params[0:3]
    # orientation matrix is inverse of transformation from lab to detector frame
    O = np.dot(rotate_x(a), np.dot(rotate_y(b), rotate_z(c))).T
    R = np.dot(rotate_x(-delta), rotate_y(gamma))
#     R = np.dot(rotate_x(gamma), rotate_y(delta)) # this is for x-dir up
    return np.dot(R, O), R


def direct_beam(params: Params, delta: float, gamma: float) -> np.ndarray:
    '''Calculate where direct beam intercepts detector in pixel coordinate'''
    O, R = orient_detector(params, delta, gamma)
    p = np.array(params[3:])
    p = np.dot(R, p)
    d = np.dot(p, O[:,2]) # distance from sample to nearest point on detector

    # direct beam is (0, 0, z)
    # thus (0, 0, z) * O[:,2] = d
    # and relative position vector needs to be transformed back to detector frame
    p_b = np.array([0, 0, d/O[2,2]]) - p
    m = np.dot(O.T, p_b)[:2]
    return m/pixel_size


def detector_distance(params: Params) -> float:
    """Return disance from sample to nearest point on detector"""
    O, R = orient_detector(params, 0, 0)
    return np.dot(params[3:], O[:, 2])


def pixels_diff(params: Params, delta_values: np.ndarray, gamma_values: np.ndarray, peak_positions: np.ndarray) -> np.ndarray:
    # data = np.asarray(args[2])  # (..., (222, 95), ...,) - ravelled = (i, j, ..., 222, 95, ..., i, j)
    peak_positions_ij = np.asarray(peak_positions)[:, ::-1]
    results = np.array([direct_beam(params, d, g) for g, d in zip(gamma_values, delta_values)])
    return results - peak_positions_ij


def residuals_sq(params: Params, delta_values: np.ndarray, gamma_values: np.ndarray, peak_positions: np.ndarray) -> float:
    ps = pixels_diff(params, delta_values, gamma_values, peak_positions)
    return np.square(ps).sum()


In [ ]:
# function for generating geometry.xml file

def generate_xml(fast_direction: np.ndarray, slow_direction: np.ndarray, origin: np.ndarray, 
                 pixel_size: float, detector_distance: float, timestamp: str, scannumber: int, detector: str) -> ET.ElementTree:
    """
    Generate xml ElementTree for detector geometry
    """
    
    def add_vector(node, *args):
        vector_node = ET.SubElement(node, 'vector')
        for value in args:
            element = ET.SubElement(vector_node, 'element')
            element.text = format_string % value
            # vector_node.append(element)
        return vector_node

    def add_node(node, name, tag, vector, size, units):
        new_node = ET.SubElement(node, tag)
        new_node.attrib['name'] = name
        vector_node = add_vector(new_node, *vector)
        size_node = ET.SubElement(new_node, "size")
        size_node.text = size
        unit_node = ET.SubElement(new_node, "units")
        unit_node.text = units
        return new_node

    def indent_tree(elem, level=0):  # see ElementTree.indent!
        indent = "\t"
        i = "\n" + level * indent
        if len(elem):
            if not elem.text or not elem.text.strip():
                elem.text = i + indent
            if not elem.tail or not elem.tail.strip():
                elem.tail = i
            for elem in elem:
                indent_tree(elem, level+1)
            if not elem.tail or not elem.tail.strip():
                elem.tail = i
        else:
            if level and (not elem.tail or not elem.tail.strip()):
                elem.tail = i

    format_string = '%.9f'
    root = ET.Element('geometry')
    det_node = ET.SubElement(root, 'detector')
    det_node.attrib['name'] = detector

    fast_node = add_node(det_node, 'fast', 'axis', fast_direction, str(pixel_size), "mm")
    slow_node = add_node(det_node, 'slow', 'axis', slow_direction, str(pixel_size), "mm")
    position_node = add_node(det_node, 'origin', 'position', origin, str(detector_distance), "mm")

    date_node = ET.SubElement(det_node, 'time')
    date_node.text = timestamp
    scan_node = ET.SubElement(det_node, 'scan')
    scan_node.text = "%d" % scannumber

    # ET.indent(root, '    ')
    indent_tree(root)
    return ET.ElementTree(root)


## Load data

In [ ]:
nxmap = hdfmap.create_nexus_map(inpath)

detector = next(iter(nxmap.image_data))
detector_path = nxmap.image_data[detector]

md_format = """
### {filename}
{start_time}

 - scan number = {entry_identifier}
 - cmd = *{scan_command}*
 - E = {incident_energy:.4} keV
 - mtthp = {mtthp:.2f} (should be 0 for merlin!)
 - delta_axis_offset = {mean(delta_axis_offset):.1f}
 - axes0 = *{_axes0}*, shape = {axes0.shape}
 - axes1 = *{_axes1}*, shape = {axes1.shape}
 - signal = *{_signal}*, shape = {signal.shape}
"""

with nxmap.load_hdf() as hdf:
    delta = nxmap.get_data(hdf,  nxmap.scannables['delta'])
    gamma = nxmap.get_data(hdf,  nxmap.scannables['gam'])
    delta_axis_offset = nxmap.get_data(hdf, 'delta_axis_offset')
    data = nxmap.get_data(hdf, detector_path)
    md = nxmap.format_hdf(hdf, md_format)
    pixel_size = nxmap.get_data(hdf, 'fast_pixel_direction')  # mm
    scannumber = nxmap.get_data(hdf, 'entry_identifier')
    start_time = str(nxmap.get_data(hdf, 'start_time'))

# Use the delta absolute angle without offset (~-8.8 deg for pilatus)
delta = delta - delta_axis_offset

md += f" - detector = '{detector}' : *{detector_path}*, shape = {data.shape}\n"
md += f" - pixel size = {pixel_size} mm\n"
display(Markdown(md))

In [ ]:
if np.mean(delta) > 5 or np.mean(gamma) > 1:
    raise Exception('Detector is not in direct beam!')
if len(np.shape(delta)) != 2 or len(np.shape(gamma)) != 2:
    raise Exception('Scan type must be 2D')
if detector.endswith('s'):
    detector = detector[:-1]  # merlins -> merlin
if detector not in ['pil3_100k', 'merlin']:
    raise Excpetion(f"Detector type not recognised: {detector}")

In [ ]:
# Interactive image slider
def view_image(index):
    plt.figure()
    with nxmap.load_hdf() as hdf:
        idx = nxmap.get_image_index(index)
        plt.title(f"delta = {delta[idx]:.2}\ngamma = {gamma[idx]:.2}")
        plt.imshow(nxmap.get_image(hdf, index))
    plt.show()


widgets.interact(view_image, index=(0, nxmap.scannables_length() - 1));

### NeXus File Tree

In [ ]:
# Display NeXus file tree
print(hdfmap.hdf_tree_string(inpath))

## Find peaks in data

In [ ]:

flat_data = np.reshape(data, (-1,) + data.shape[-2:]).astype(int)  # reshape volume to [i, j, k], where i is the scan direction
flat_delta = np.reshape(delta, -1)
flat_gamma = np.reshape(gamma, -1)

# Find peak in each image
peaks = peak_search(flat_data)

# Plot peak positions
for d1 in range(1 + len(flat_data)//9):
    plt.figure(figsize=[16, 10], dpi=100)
    for d2 in range(9):
        n = 9*d1 + d2
        if n >= len(flat_data):
            break
        plt.subplot(3,3,d2+1)
        #plt.imshow(data[n, 200:500, 0:300])
        plt.imshow(flat_data[n])
        plt.plot([peaks[n][1]], [peaks[n][0]], 'w+', ms=16, lw=2)
        # plt.clim([0, 10])
        plt.title('delta=%5.3f, gamma=%5.3f' % (flat_delta[n], flat_gamma[n]))
        plt.xlim([peaks[n][1] - 20, peaks[n][1] + 20])
        plt.ylim([peaks[n][0] - 20, peaks[n][0] + 20])

In [ ]:
# Plot positions
plt.figure()
plt.title(f"Peak Positions on {detector} detector")
plt.plot(peaks[:, 0], peaks[:, 1], 'k-+')


## Fit Detector geometry using peak positions

In [ ]:

# Initial parameters
if detector == 'merlin':
    initial_params = Params(x_rot=0, y_rot=0, z_rot=0, x_pos=-9, y_pos=-19, z_pos=1300)
else:
    initial_params = Params(x_rot=0.5, y_rot=-45, z_rot=0, x_pos=18, y_pos=50, z_pos=500)
bounds = [(-180, 180), (-180, 180), (-180, 180), (-1000, 1000), (-1000, 1000), (0, 2000)]

# Perform fit
#fitted_params = calibrate_detector2(data, deltas, gammas, initial_params)
fitted_params = optimize.fmin_l_bfgs_b(
    residuals_sq, 
    initial_params, 
    bounds=bounds, 
    args=(flat_delta, flat_gamma, peaks), 
    approx_grad=True
)

params = Params(*fitted_params[0])
param_string = '\n'.join(f" - {key:5}: {val:.2f}" for key, val in params._asdict().items())
residual = residuals_sq(params, flat_delta, flat_gamma, peaks)

result = f"""
### Fit Finished
**Fitted parameters**
{param_string}

**Residual^2**: {residual:.4}

### Compare fitted peak positions
"""
display(Markdown(result))

print('delta  gamma  Meas. peak    Model peak        Difference')
for n in range(len(peaks)):
    pk = direct_beam(fitted_params[0], flat_delta[n], flat_gamma[n])
    # print('%6.2f %6.2f %s    %s' % (flat_delta[n], flat_gamma[n], peaks[n], pk))
    diff = np.sqrt(np.sum(np.square(peaks[n] - pk[::-1])))
    print(f"{flat_delta[n]:6.2f} {flat_gamma[n]:6.2f} {str(peaks[n]):10}   ({pk[1]:6.2f}, {pk[0]:6.2f})   {diff:.2f}")


## Detector Geometry

#### Lab-frame geometry
 - **X-Axis**: horizontal, away from ring
 - **Y-Axis**: vertical, towards ceiling
 - **Z-Axis**: along beam direction

In [ ]:
date = datetime.datetime.now().strftime('%Y_%m_%d')

orientation_matrix = orient_detector(params)[0]
fast = orientation_matrix.T[0]  # lab-frame x-axis (vertical, towards ceiling)
slow = orientation_matrix.T[1]  # lab-frame y-axis (horizontal, towards ring)
normal = orientation_matrix.T[2]  # lab-frame z-axis (along beam direction)

detector_shape = data.shape[-2:]
detector_origin_position = fitted_params[0][3:]
detector_origin_size = np.sqrt(np.sum(np.square(detector_origin_position)))
detector_origin_norm = detector_origin_position / detector_origin_size

detector_centre_position = detector_origin_position + (fast * pixel_size * detector_shape[1]/2) + (slow * pixel_size * detector_shape[0]/2)
detector_centre_size = np.sqrt(np.sum(np.square(detector_centre_position)))

print(f"Detector fast-axis: {fast}, {detector_shape[1]} pixels, {detector_shape[1] * pixel_size:.2f} mm")
print(f"Detector slow-axis: {slow}, {detector_shape[0]} pixels, {detector_shape[0] * pixel_size:.2f} mm")
print(f"   Detector normal: {normal}")

print(f"\nDetector(origin)-sample distance: {detector_origin_size:2f} mm")
print(f"Detector(centre)-sample distance: {detector_centre_size:2f} mm")


In [ ]:
fig = plt.figure(figsize=[18, 6], dpi=80)
ttl = f"{detector} detector geometry"
plt.suptitle(ttl)

beam = np.array([
    [0, 0, 0],
    detector_centre_position
])
det = np.array([
    detector_origin_position,
    detector_origin_position + fast * pixel_size * detector_shape[1],
    detector_origin_position + fast * pixel_size * detector_shape[1] + slow * pixel_size * detector_shape[0],
    detector_origin_position + slow * pixel_size * detector_shape[0],
    detector_origin_position
])
    

ax = fig.add_subplot(131, projection='3d')
ax.plot(beam[:, 0], beam[:, 2], beam[:, 1], 'r-', lw=2)
ax.plot(det[:, 0], det[:, 2], det[:, 1], 'k-', ms=16)
ax.plot([detector_origin_position[0]], [detector_origin_position[2]], [detector_origin_position[1]], 'k+', ms=16)
scale = 1.1 * max(detector_shape) * pixel_size / 2
ax.set_zlim(detector_centre_position[1] + [-scale, scale])
ax.set_xlim(detector_centre_position[0] + [-scale, scale])
ax.set_xlabel('X')
ax.set_ylabel('Z')
ax.set_zlabel('Y')

ax = fig.add_subplot(132)
ax.plot(beam[:, 2], beam[:, 0], 'r-', lw=2)
ax.plot(det[:, 2], det[:, 0], 'k-', ms=16)
ax.plot([detector_origin_position[2]], [detector_origin_position[0]], 'k+', ms=16)
ax.set_xlabel('Z')
ax.set_ylabel('X')
ax.axis('image')

ax = fig.add_subplot(133)
ax.plot(beam[:, 1], beam[:, 2], 'r-', lw=2)
ax.plot(det[:, 1], det[:, 2], 'k-', ms=16)
ax.plot([detector_origin_position[1]], [detector_origin_position[2]], 'k+', ms=16)
ax.set_xlabel('Y')
ax.set_ylabel('Z')
ax.axis('image')


## Generate Geometry XML file

In [ ]:
xml_tree = generate_xml(
    fast_direction=fast, 
    slow_direction=slow, 
    origin=detector_origin_norm, 
    pixel_size=pixel_size, 
    detector_distance=detector_origin_size, 
    timestamp=start_time, 
    scannumber=scannumber, 
    detector=detector
)

calibration_dir = "/dls_sw/i16/scripts/detector_calibration/"
filename_backup = f"old_calibration_files/geometry_{detector}_{date}.xml"
filename = f"geometry_{detector}.xml"

# Save files
xml_tree.write(calibration_dir + filename_backup)
xml_tree.write(calibration_dir + filename)
print(f"Saved {detector} calibration files:\n  {calibration_dir + filename}\n  {calibration_dir + filename_backup} (backup)")

In [ ]:
print(calibration_dir + filename_backup)
with open(calibration_dir + filename_backup) as f:
    print(f.read())